In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/README.roboflow.txt
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/DJI_0034_JPG.rf.afad2e63a2d68a3999464dcb26ded149.jpg
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/20240120_162850_jpg.rf.39a5cd396bb8f99c71916eb18d3e0297.jpg
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/20240120_164348_mp4-4722_jpg.rf.ca40a6bd04e7d0a50ebbc618bfc0d7a3.jpg
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/20240120_163723-0-_jpg.rf.b764b5145b91bcc084f7a124b29f1d20.jpg
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/20240120_162836_jpg.rf.28c044cc697a91a494596b4fc544b4b9.jpg
/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/20240120_163001-0-_jpg.rf.4d3f

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [6]:
import os
import shutil
import json
from pathlib import Path
from tqdm import tqdm
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

In [4]:
# TRAIN
input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train'
output_dir = '/kaggle/working/dataset/train/images'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(output_dir, filename))

input_annotation_file = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
output_annotation_file = '/kaggle/working/dataset/train_annotations.json'
shutil.copy(input_annotation_file, output_annotation_file)

'/kaggle/working/dataset/train_annotations.json'

In [5]:
# VALID
input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid'
output_dir = '/kaggle/working/dataset/valid/images'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(output_dir, filename))

input_annotation_file = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json'
output_annotation_file = '/kaggle/working/dataset/valid_annotations.json'
shutil.copy(input_annotation_file, output_annotation_file)

'/kaggle/working/dataset/valid_annotations.json'

In [8]:
# TEST
input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/test'
output_dir = '/kaggle/working/dataset/test/images'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(output_dir, filename))

In [19]:
RESCALE_SIZE = 2048
PATCH_SIZE = 256
STEP_SIZE = 256
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#OUTPUT_PATCHES_DIR = Path("/kaggle/working/patch_dataset")
#OUTPUT_PATCHES_DIR.mkdir(parents=True, exist_ok=True)
#(OUTPUT_PATCHES_DIR / "images").mkdir(exist_ok=True)
#(OUTPUT_PATCHES_DIR / "labels").mkdir(exist_ok=True)

In [10]:
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size + 1, step_size):
        for x in range(0, image.shape[1] - window_size + 1, step_size):
            yield (x, y, image[y:y + window_size, x:x + window_size])


def extract_between_last_dots(s):
    return '.'.join(s.split('.')[:-1])

def create_annotations_dict(coco_data):
    images_id = {
        a['id']: {
            'file_name': a['file_name'],
            'width': a['width'],
            'height': a['height']
        } for a in coco_data['images']
    }

    images_damages = {}

    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        file_key = extract_between_last_dots(images_id[image_id]['file_name'])

        if file_key not in images_damages:
            images_damages[file_key] = []

        # Преобразование category_id из диапазона 1-5 в диапазон 0-4
        cat_id = ann['category_id'] - 1

        images_damages[file_key].append({
            'cat_id': cat_id,
            'box': ann['bbox'],
            'width': images_id[image_id]['width'],
            'height': images_id[image_id]['height']
        })

    return images_damages

def box_in_patch(bbox, patch_coords, patch_size):
    x_min, y_min, width, height = bbox
    bbox_xmax = x_min + width
    bbox_ymax = y_min + height

    px, py = patch_coords
    patch_xmax = px + patch_size
    patch_ymax = py + patch_size

    inter_xmin = max(x_min, px)
    inter_ymin = max(y_min, py)
    inter_xmax = min(bbox_xmax, patch_xmax)
    inter_ymax = min(bbox_ymax, patch_ymax)

    if inter_xmax <= inter_xmin or inter_ymax <= inter_ymin:
        return None

    new_xmin = inter_xmin - px
    new_ymin = inter_ymin - py
    new_xmax = inter_xmax - px
    new_ymax = inter_ymax - py

    return [new_xmin, new_ymin, new_xmax, new_ymax]

In [11]:
class DetectionPatchesDataset(Dataset):
    def __init__(self, image_paths, annotations_dict, transform=None):
        self.image_paths = image_paths
        self.annotations = annotations_dict
        self.transform = transform
        self.patch_infos = []

        for image_path in tqdm(self.image_paths, desc="Preparing patches"):
            image_name = image_path.stem
            image = Image.open(image_path).convert("RGB")
            image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
            image_np = np.array(image)

            anns = self.annotations.get(image_name, [])

            for (x, y, patch) in sliding_window(image_np, STEP_SIZE, PATCH_SIZE):
                patch_boxes = []
                patch_labels = []

                for ann in anns:
                    mapped_box = box_in_patch(ann['box'], (x, y), PATCH_SIZE)
                    if mapped_box:
                        patch_boxes.append(mapped_box)
                        patch_labels.append(ann['cat_id'])

                if patch_boxes:
                    self.patch_infos.append({
                        'image_path': image_path,
                        'patch_coords': (x, y),
                        'boxes': patch_boxes,
                        'labels': patch_labels
                    })

    def __len__(self):
        return len(self.patch_infos)

    def __getitem__(self, idx):
        info = self.patch_infos[idx]
        image = Image.open(info['image_path']).convert("RGB")
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        image_np = np.array(image)

        x, y = info['patch_coords']
        patch = image_np[y:y+PATCH_SIZE, x:x+PATCH_SIZE]

        boxes = np.array(info['boxes'], dtype=np.float32)
        labels = np.array(info['labels'], dtype=np.int64)

        if self.transform:
            transformed = self.transform(image=patch, bboxes=boxes, class_labels=labels)
            patch = transformed['image']
            boxes = torch.tensor(transformed['bboxes'], dtype=torch.float32)
            labels = torch.tensor(transformed['class_labels'], dtype=torch.int64)
        else:
            patch = ToTensor()(patch)
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels
        }

        return patch, target

In [13]:
def convert_coco_to_yolo(annotations, image_dir, label_dir):
    for image_file in tqdm(os.listdir(image_dir), desc="Converting annotations"):
        if image_file.endswith('.jpg'):
            image_path = os.path.join(image_dir, image_file)
            image = Image.open(image_path)
            width, height = image.size
            label_path = os.path.join(label_dir, image_file.replace('.jpg', '.txt'))

            with open(label_path, 'w') as f:
                image_id = extract_between_last_dots(image_file)
                if image_id in annotations:
                    anns = annotations[image_id]
                    for ann in anns:
                        cat_id = ann['cat_id']
                        bbox = ann['box']
                        x_center = (bbox[0] + bbox[2] / 2) / width
                        y_center = (bbox[1] + bbox[3] / 2) / height
                        bbox_width = bbox[2] / width
                        bbox_height = bbox[3] / height
                        f.write(f"{cat_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                else:
                    # Write an empty line for no defect
                    f.write("")

In [ ]:
def save_patches_and_labels(image_paths, annotations_dict, prefix="train"):
    for image_path in tqdm(image_paths, desc=f"Processing {prefix} images"):
        image_name = image_path.stem
        anns = annotations_dict.get(image_name, [])

        image = Image.open(image_path).convert("RGB")
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        image_np = np.array(image)

        for (x, y, patch) in sliding_window(image_np, STEP_SIZE, PATCH_SIZE):
            patch_boxes = []
            patch_labels = []

            for ann in anns:
                mapped_box = box_in_patch(ann['box'], (x, y), PATCH_SIZE)
                if mapped_box:
                    patch_boxes.append(mapped_box)
                    patch_labels.append(ann['cat_id'])

            if patch_boxes:
                # Сохраняем патч
                patch_img = Image.fromarray(patch)
                patch_name = f"{prefix}_{image_name}_x{x}_y{y}"
                img_save_path = OUTPUT_PATCHES_DIR / "images" / f"{patch_name}.jpg"
                lbl_save_path = OUTPUT_PATCHES_DIR / "labels" / f"{patch_name}.txt"
                patch_img.save(img_save_path)

                with open(lbl_save_path, 'w') as f:
                    for bbox, label in zip(patch_boxes, patch_labels):
                        x_min, y_min, x_max, y_max = bbox
                        xc = (x_min + x_max) / 2 / PATCH_SIZE
                        yc = (y_min + y_max) / 2 / PATCH_SIZE
                        bw = (x_max - x_min) / PATCH_SIZE
                        bh = (y_max - y_min) / PATCH_SIZE
                        f.write(f"{label} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

In [14]:
train_annot_path = '/kaggle/working/dataset/train_annotations.json'
val_annot_path = '/kaggle/working/dataset/valid_annotations.json'

with open(train_annot_path, 'r') as f:
    train_coco = json.load(f)
with open(val_annot_path, 'r') as f:
    val_coco = json.load(f)

train_annotations = create_annotations_dict(train_coco)
val_annotations = create_annotations_dict(val_coco)

In [15]:
train_image_paths = list(Path('/kaggle/working/dataset/train/images').glob('*.jpg'))
val_image_paths = list(Path('/kaggle/working/dataset/valid/images').glob('*.jpg'))

train_dataset = DetectionPatchesDataset(train_image_paths, train_annotations, transform=None)
val_dataset = DetectionPatchesDataset(val_image_paths, val_annotations, transform=None)

Preparing patches: 100%|██████████| 271/271 [01:14<00:00,  3.63it/s]


In [16]:
# Convert COCO annotations to YOLO format
os.makedirs('/kaggle/working/dataset/train/labels', exist_ok=True)
os.makedirs('/kaggle/working/dataset/valid/labels', exist_ok=True)

convert_coco_to_yolo(train_annotations, '/kaggle/working/dataset/train/images', '/kaggle/working/dataset/train/labels')
convert_coco_to_yolo(val_annotations, '/kaggle/working/dataset/valid/images', '/kaggle/working/dataset/valid/labels')

Converting annotations: 100%|██████████| 271/271 [00:00<00:00, 5081.98it/s]


In [17]:
# Создание data.yaml для YOLO
data_yaml = """
train: /kaggle/working/dataset/train/images
val: /kaggle/working/dataset/valid/images

nc: 5
names: ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']
"""

with open('/kaggle/working/dataset/data.yaml', 'w') as f:
    f.write(data_yaml)

model = YOLO('yolov8s.yaml')  # nano-модель
model.model.args['num_classes'] = 5
model.model.names = [
    'corrosion',
    'lightning',
    'lightning receptor',
    'missing teeth',
    'patch'
]

In [18]:
config = {
    'data': '/kaggle/working/dataset/data.yaml',
    'epochs': 10,
    'imgsz': PATCH_SIZE,
    'batch': 4,
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.2,
    'copy_paste': 0.2,
    'degrees': 15,
    'shear': 0.1,
    'perspective': 0.0001,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.5,
    'dropout': 0.1
}

# -------- Запуск обучения --------
model.train(**config)

Ultralytics 8.3.120 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/dataset/data.yaml, epochs=10, time=None, patience=100, batch=4, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

100%|██████████| 755k/755k [00:00<00:00, 17.7MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /kaggle/working/dataset/train/labels... 312 images, 221 backgrounds, 0 corrupt:  29%|██▉       | 312/1070 [00:00<00:00, 1334.65it/s]/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (149817600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
train: Scanning /kaggle/working/dataset/train/labels... 1070 images, 316 backgrounds, 0 corrupt: 100%|██████████| 1070/1070 [00:00<00:00, 1185.55it/s]

train: New cache created: /kaggle/working/dataset/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1676.4±894.4 MB/s, size: 806.6 KB)


...

       8/10         0G      1.196      1.686     0.3747          0        256: 100%|██████████| 268/268 [02:57<00:00,  1.51it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:24<00:00,  1.38it/s]

                   all        271       2619          0          0          0          0



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       9/10         0G      1.146      1.696     0.3014          3        256:   3%|▎         | 9/268 [00:05<02:42,  1.60it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_annotations(image_path, annotations):
    image = Image.open(image_path).convert("RGB")
    plt.imshow(image)
    ax = plt.gca()

    image_name = Path(image_path).stem
    if image_name in annotations:
        anns = annotations[image_name]
        for ann in anns:
            bbox = ann['box']
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            plt.text(bbox[0], bbox[1], f"Class {ann['cat_id']}", color='red', fontsize=12, backgroundcolor='white')

    plt.show()

# Пример использования
image_path = '/kaggle/working/dataset/train/images/20240120_150724_jpg.rf.d10477a0552d30811fc800c5843e81d2.jpg'
visualize_annotations(image_path, train_annotations)

# YOLO v8

In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import json
from pathlib import Path
from tqdm import tqdm
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

In [ ]:
# TRAIN
input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train'
output_dir = '/kaggle/working/dataset/train/images'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(output_dir, filename))

input_annotation_file = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json'
output_annotation_file = '/kaggle/working/dataset/train_annotations.json'
shutil.copy(input_annotation_file, output_annotation_file)

In [ ]:
# VALID
input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid'
output_dir = '/kaggle/working/dataset/valid/images'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(output_dir, filename))

input_annotation_file = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json'
output_annotation_file = '/kaggle/working/dataset/valid_annotations.json'
shutil.copy(input_annotation_file, output_annotation_file)

In [ ]:
# TEST
input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/test'
output_dir = '/kaggle/working/dataset/test/images'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(output_dir, filename))

In [5]:
# Параметры
PATCH_SIZE = 256
STEP_SIZE = 256
RESCALE_SIZE = 2048
OUTPUT_PATCHES_DIR = Path("/kaggle/working/patch_dataset")
OUTPUT_PATCHES_DIR.mkdir(parents=True, exist_ok=True)
(OUTPUT_PATCHES_DIR / "images").mkdir(exist_ok=True)
(OUTPUT_PATCHES_DIR / "labels").mkdir(exist_ok=True)

def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size + 1, step_size):
        for x in range(0, image.shape[1] - window_size + 1, step_size):
            yield (x, y, image[y:y + window_size, x:x + window_size])

def box_in_patch(bbox, patch_coords, patch_size):
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height

    px, py = patch_coords
    patch_xmax = px + patch_size
    patch_ymax = py + patch_size

    inter_xmin = max(x_min, px)
    inter_ymin = max(y_min, py)
    inter_xmax = min(x_max, patch_xmax)
    inter_ymax = min(y_max, patch_ymax)

    if inter_xmax <= inter_xmin or inter_ymax <= inter_ymin:
        return None

    # Новые координаты внутри патча
    new_xmin = inter_xmin - px
    new_ymin = inter_ymin - py
    new_xmax = inter_xmax - px
    new_ymax = inter_ymax - py

    return [new_xmin, new_ymin, new_xmax, new_ymax]

def extract_between_last_dots(s):
    return '.'.join(s.split('.')[:-1])

def create_annotations_dict(coco_data):
    images_id = {
        a['id']: {
            'file_name': a['file_name'],
            'width': a['width'],
            'height': a['height']
        } for a in coco_data['images']
    }

    images_damages = {}

    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        file_key = extract_between_last_dots(images_id[image_id]['file_name'])

        if file_key not in images_damages:
            images_damages[file_key] = []

        images_damages[file_key].append({
            'cat_id': ann['category_id'] - 1,  # YOLO формат: от 0 до X, в аннотациях от 1 до X
            'box': ann['bbox'],
            'width': images_id[image_id]['width'],
            'height': images_id[image_id]['height']
        })

    return images_damages

def save_patches_and_labels(image_paths, annotations_dict, prefix):
    (OUTPUT_PATCHES_DIR / f"{prefix}" / "images").mkdir(parents=True, exist_ok=True)
    (OUTPUT_PATCHES_DIR / f"{prefix}" / "labels").mkdir(parents=True, exist_ok=True)
    for image_path in tqdm(image_paths, desc=f"Processing {prefix} images"):
        image_name = image_path.stem
        anns = annotations_dict.get(image_name, [])

        image = Image.open(image_path).convert("RGB")
        #image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        image_np = np.array(image)

        for (x, y, patch) in sliding_window(image_np, STEP_SIZE, PATCH_SIZE):
            patch_boxes = []
            patch_labels = []

            for ann in anns:
                mapped_box = box_in_patch(ann['box'], (x, y), PATCH_SIZE)
                if mapped_box:
                    patch_boxes.append(mapped_box)
                    patch_labels.append(ann['cat_id'])

            if patch_boxes:
                patch_img = Image.fromarray(patch)
                patch_name = f"{prefix}_{image_name}_x{x}_y{y}"
                img_save_path = OUTPUT_PATCHES_DIR / f"{prefix}" / "images" / f"{patch_name}.jpg"
                lbl_save_path = OUTPUT_PATCHES_DIR / f"{prefix}" / "labels" / f"{patch_name}.txt"
                patch_img.save(img_save_path)

                with open(lbl_save_path, 'w') as f:
                    for bbox, label in zip(patch_boxes, patch_labels):
                        x_min, y_min, x_max, y_max = bbox
                        xc = (x_min + x_max) / 2 / PATCH_SIZE
                        yc = (y_min + y_max) / 2 / PATCH_SIZE
                        bw = (x_max - x_min) / PATCH_SIZE
                        bh = (y_max - y_min) / PATCH_SIZE
                        f.write(f"{label} {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}\n")

if __name__ == "__main__":
    JSON_PATH = Path("/kaggle/working/dataset/train_annotations.json")
    IMAGE_DIR = Path("/kaggle/working/dataset/train/images")

    with open(JSON_PATH) as f:
        coco_data = json.load(f)

    annotations_dict = create_annotations_dict(coco_data)

    image_paths = list(IMAGE_DIR.glob("*.jpg"))

    save_patches_and_labels(image_paths, annotations_dict, prefix="train")

    JSON_PATH = Path("/kaggle/working/dataset/valid_annotations.json")
    IMAGE_DIR = Path("/kaggle/working/dataset/valid/images")

    with open(JSON_PATH) as f:
        coco_data = json.load(f)

    annotations_dict = create_annotations_dict(coco_data)

    image_paths = list(IMAGE_DIR.glob("*.jpg"))

    save_patches_and_labels(image_paths, annotations_dict, prefix="valid")


Processing valid images: 100%|██████████| 271/271 [00:31<00:00,  8.63it/s]


In [6]:
nc: 5
names: ['corrosion', 'lightning', 'lightning receptor', 'missing teeth', 'patch']


data_yaml = """
train: /kaggle/working/patch_dataset/train/images
val: /kaggle/working/patch_dataset/valid/images

nc: 5
names:
  0: corrosion
  1: lightning
  2: lightning receptor
  3: missing teeth
  4: patch
"""

with open('/kaggle/working/new_data.yaml', 'w') as f:
    f.write(data_yaml)

In [8]:
config = {
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'fliplr': 0.5,
    'degrees': 15,
    'shear': 0.1,
    'perspective': 0.0001,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.2
}

model = YOLO("yolov8s.pt")

model.train(data="/kaggle/working/new_data.yaml", epochs=50, imgsz=256, batch=8, **config)

Ultralytics 8.3.120 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/new_data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wid

train: Scanning /kaggle/working/patch_dataset/train/labels.cache... 5303 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5303/5303 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 220.6±165.7 MB/s, size: 6.0 KB)


val: Scanning /kaggle/working/patch_dataset/valid/labels.cache... 1302 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1302/1302 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 82/82 [00:06<00:00, 11.87it/s]


                   all       1302       2969      0.636      0.418      0.476      0.272
             corrosion        871       2377      0.672      0.429       0.46      0.207
             lightning         24         24      0.717     0.0833      0.134     0.0796
    lightning receptor         12         12      0.302     0.0833      0.234       0.13
         missing teeth        348        480      0.786      0.613      0.676      0.342
                 patch         47         76      0.704      0.882      0.875      0.599


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.0ms preprocess, 1.1ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7e507d510d90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

In [12]:
!pip install ensemble-boxes

In [33]:
print(model.names)

{0: 'corrosion', 1: 'lightning', 2: 'lightning receptor', 3: 'missing teeth', 4: 'patch'}


In [ ]:
import numpy as np
from pathlib import Path
from PIL import Image
import pandas as pd
from tqdm import tqdm
from ensemble_boxes import weighted_boxes_fusion

from ultralytics import YOLO

# Параметры
PATCH_SIZE = 256
STEP_SIZE = 256
IOU_THRESH = 0.01
CONF_THRESH = 0.005
TEST_IMAGE_DIR = Path("/kaggle/working/dataset/test/images")
model = YOLO("/kaggle/input/yolo/other/default/1/best(2).pt")

def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size + 1, step_size):
        for x in range(0, image.shape[1] - window_size + 1, step_size):
            yield (x, y, image[y:y + window_size, x:x + window_size])

# Предсказания для тестовых изображений
results = []

for image_path in tqdm(list(TEST_IMAGE_DIR.glob("*.jpg")), desc="Predicting"):
    image = Image.open(image_path)
    image_np = np.array(image)
    ih, iw = image_np.shape[:2]

    boxes = []
    scores = []
    labels = []

    for (x, y, patch) in sliding_window(image_np, STEP_SIZE, PATCH_SIZE):
        patch_image = Image.fromarray(patch)
        prediction = model(patch_image)[0]
        
        for box in prediction.boxes.data.cpu().numpy():
            x1, y1, x2, y2, conf, class_id = box 
            if conf < CONF_THRESH:
                continue

            # Смещаем координаты на положение патча и нормализуем для WBF
            x1_adj = (x1 + x) / iw
            y1_adj = (y1 + y) / ih
            x2_adj = (x2 + x) / iw
            y2_adj = (y2 + y) / ih

            boxes.append([x1_adj, y1_adj, x2_adj, y2_adj])
            scores.append(conf)
            labels.append(int(class_id))
            print(labels)
            
    if boxes:
        boxes_fused, scores_fused, labels_fused = weighted_boxes_fusion(
            [boxes], [scores], [labels], iou_thr=IOU_THRESH, skip_box_thr=CONF_THRESH
        )

        final_objects = []
        for box, score, label in zip(boxes_fused, scores_fused, labels_fused):
            x1, y1, x2, y2 = box
            xc = (x1 + x2) / 2 * iw
            yc = (y1 + y2) / 2 * ih
            w = (x2 - x1) * iw
            h = (y2 - y1) * ih
            final_objects.append([
                model.names[int(label)],
                float(xc),
                float(yc),
                float(w),
                float(h),
                float(score)
            ])
    else:
        final_objects = []

    results.append([image_path.stem, final_objects])

# Сохранение
df = pd.DataFrame(results, columns=["image_id", "objects"])
df.to_csv("/kaggle/working/predictions.csv", index=False)
df

In [35]:
import numpy as np
from pathlib import Path
from PIL import Image
import pandas as pd
from tqdm import tqdm

# Параметры для патчей
PATCH_SIZE = 256
STEP_SIZE = 256
TEST_IMAGE_DIR = Path("/kaggle/working/dataset/test/images")
OUTPUT_PATCHES_DIR = Path("/kaggle/working/patch_predictions")

# Загрузка модели
from ultralytics import YOLO
model = YOLO("/kaggle/working/runs/detect/train2/weights/best.pt")

# Функция для скользящего окна
def sliding_window(image, step_size, window_size):
    for y in range(0, image.shape[0] - window_size + 1, step_size):
        for x in range(0, image.shape[1] - window_size + 1, step_size):
            yield (x, y, image[y:y + window_size, x:x + window_size])

# Функция для объединения предсказаний
def merge_predictions(predictions, image_shape, patch_size, step_size):
    # Создаём пустое изображение для объединённых предсказаний
    merged = np.zeros(image_shape)
    
    for (x, y, pred) in predictions:
        # Наложение предсказания на соответствующую область изображения
        merged[y:y+patch_size, x:x+patch_size] = pred  # Здесь, например, добавляем метки объектов
    
    return merged

# Предсказания для тестовых изображений
results = []

# Процесс предсказания для каждого изображения
for image_path in tqdm(list(TEST_IMAGE_DIR.glob("*.jpg")), desc="Predicting"):
    # Открытие изображения
    image = Image.open(image_path)
    image_np = np.array(image)

    # Получение предсказаний для каждого патча
    predictions = []
    for (x, y, patch) in sliding_window(image_np, STEP_SIZE, PATCH_SIZE):
        patch_image = Image.fromarray(patch)
        prediction = model(patch_image)  # Предсказание для патча
        objects = []

        for pred in prediction[0].boxes.data.tolist():
            class_id, x_center, y_center, width, height, confidence = pred
            print(class_id, x_center, y_center, width, height, confidence)
            label = model.names[int(class_id)]
            objects.append([label, x_center.item(), y_center.item(), width.item(), height.item(), confidence.item()])

        if objects:
            predictions.append((x, y, objects))

    # Объединение результатов в одно изображение
    merged_result = merge_predictions(predictions, image_np.shape, PATCH_SIZE, STEP_SIZE)

    # Формирование final result для сохранения
    image_id = image_path.stem
    results.append([image_id, predictions])

# Сохранение в CSV файл
df = pd.DataFrame(results, columns=["image_id", "objects"])
df.to_csv("/kaggle/working/predictions1.csv", index=False)
df

Predicting:   0%|          | 0/151 [00:00<?, ?it/s]


0: 256x256 (no detections), 8.7ms
Speed: 0.6ms preprocess, 8.7ms inference, 0.8ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.0ms
Speed: 0.5ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.1ms
Speed: 0.4ms preprocess, 7.1ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.0ms
Speed: 0.4ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.2ms
Speed: 0.4ms preprocess, 7.2ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.0ms
Speed: 0.4ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.0ms
Speed: 0.4ms preprocess, 7.0ms inference, 0.6ms postprocess per image at shape (1, 3, 256, 256)

0: 256x256 (no detections), 7.0ms
Speed: 0.4ms preprocess, 7.0ms inference, 0.6ms 

Predicting:   0%|          | 0/151 [00:00<?, ?it/s]

97.60855102539062 0.0 104.56817626953125 6.914254188537598 0.413756787776947 0.0


KeyError: 97

In [16]:
df['image_id'] = df['image_id'].apply(lambda x: x + ".jpg")

def correct_json_format(objects):
    objects_list = eval(objects)
    # Преобразуем список в строку JSON с двойными кавычками
    corrected_objects = json.dumps(objects_list)
    return corrected_objects

# Исправление формата данных
df['objects'] = df['objects'].apply(correct_json_format)

df.to_csv("/kaggle/working/predictions.csv", index=False)
df

,image_id,objects
0,DJI_0703-1-_JPG.rf.f37e56bcbcea4be50fbb6916395...,"[[corrosion, 2497.167999267578, 1681.138198852..."
1,DJI_0766-1-_JPG.rf.23aa70af0a4a9b9557902b42079...,"[[corrosion, 726.4273223876953, 1512.367599487..."
2,20240120_161207_jpg.rf.b811af46b8e7c338c1ffaab...,"[[corrosion, 2275.8599224090576, 1129.70973563..."
3,DJI_0786_JPG.rf.8393511dd03727861777c834fdfd3d...,"[[corrosion, 2029.6153869628909, 1323.67815780..."
4,20240120_162844-0-_jpg.rf.9b6edc74a2607728b200...,"[[missing teeth, 1464.8779296875, 2322.5268130..."
...,...,...
146,20240120_153748-0-_jpg.rf.6c50a8089bf3ccd32bb6...,"[[corrosion, 3190.9919052124023, 2490.34487533..."
147,20240120_164348_mp4-2330_jpg.rf.99f885a1881dee...,"[[missing teeth, 658.3693237304686, 159.574966..."
148,DJI_0782_JPG.rf.20122d86e06af6481889f80a374812...,"[[corrosion, 904.1289825439454, 2715.738830566..."
149,20240120_160553_jpg.rf.02cafabaceab373ddc1e5c3...,"[[corrosion, 859.7668762207031, 2464.085098266..."


# YOLO v11

In [ ]:
!pip install ultralytics

In [2]:
import os
import shutil
import json
from pathlib import Path
from tqdm import tqdm
import numpy as np
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor

from albumentations.pytorch import ToTensorV2
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.6' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# Параметры
RESCALE_SIZE = 1024

def extract_between_last_dots(s):
    return '.'.join(s.split('.')[:-1])

def create_annotations_dict(coco_data):
    images_id = {
        a['id']: {
            'file_name': a['file_name'],
            'width': a['width'],
            'height': a['height']
        } for a in coco_data['images']
    }

    images_damages = {}

    for ann in coco_data['annotations']:
        image_id = ann['image_id']
        file_key = extract_between_last_dots(images_id[image_id]['file_name'])

        if file_key not in images_damages:
            images_damages[file_key] = []

        images_damages[file_key].append({
            'cat_id': ann['category_id'] - 1,  # YOLO формат: от 0 до X, в аннотациях от 1 до X
            'box': ann['bbox'],
            'width': images_id[image_id]['width'],
            'height': images_id[image_id]['height']
        })

    return images_damages

def save_yolo_labels_from_coco(images, annotations_dict, label_dir):
    """
    Преобразует COCO-аннотации в YOLO формат для исходных изображений.
    """
    for image_path in tqdm(images, desc=f"Creating YOLO labels for {label_dir.name}"):
        image_name = image_path.stem
        anns = annotations_dict.get(image_name, [])
        label_file = label_dir / f"{image_name}.txt"

        with open(label_file, "w") as f:
            for ann in anns:
                x_min, y_min, width, height = ann['box']
                img_w, img_h = ann['width'], ann['height']
                x_center = (x_min + width / 2) / img_w
                y_center = (y_min + height / 2) / img_h
                w = width / img_w
                h = height / img_h
                f.write(f"{ann['cat_id']} {x_center} {y_center} {w} {h}\n")

def move_images_to_subdir(input_dir, subfolder="images"):
    input_dir = Path(input_dir)
    output_dir = input_dir / subfolder
    output_dir.mkdir(exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith('.jpg'):
            img_path = input_dir / filename
            img = Image.open(img_path)
            img.save(output_dir / filename)

    return output_dir

In [4]:
# === Копирование изображений ===

# TRAIN
train_input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train'
train_output_dir = '/kaggle/working/dataset/train/images'
os.makedirs(train_output_dir, exist_ok=True)

for filename in os.listdir(train_input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(train_input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(train_output_dir, filename))

# VALID
valid_input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid'
valid_output_dir = '/kaggle/working/dataset/valid/images'
os.makedirs(valid_output_dir, exist_ok=True)

for filename in os.listdir(valid_input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(valid_input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(valid_output_dir, filename))

# TEST
test_input_dir = '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/test'
test_output_dir = '/kaggle/working/dataset/test/images'
os.makedirs(test_output_dir, exist_ok=True)

for filename in os.listdir(test_input_dir):
    if filename.endswith('.jpg'):
        img_path = os.path.join(test_input_dir, filename)
        img = Image.open(img_path)
        img.save(os.path.join(test_output_dir, filename))


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:3402: DecompressionBombWarning: Image size (149817600 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [5]:
# TRAIN ANNOTATIONS
shutil.copy(
    '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json',
    '/kaggle/working/dataset/train_annotations.json'
)
with open('/kaggle/working/dataset/train_annotations.json') as f:
    coco_data = json.load(f)

annotations_dict = create_annotations_dict(coco_data)
train_labels_dir = Path('/kaggle/working/dataset/train/labels')
train_labels_dir.mkdir(parents=True, exist_ok=True)
save_yolo_labels_from_coco(list(Path('/kaggle/working/dataset/train/images').glob("*.jpg")), annotations_dict, train_labels_dir)

# VALID ANNOTATIONS
shutil.copy(
    '/kaggle/input/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json',
    '/kaggle/working/dataset/valid_annotations.json'
)
with open('/kaggle/working/dataset/valid_annotations.json') as f:
    coco_data = json.load(f)

annotations_dict = create_annotations_dict(coco_data)
valid_labels_dir = Path('/kaggle/working/dataset/valid/labels')
valid_labels_dir.mkdir(parents=True, exist_ok=True)
save_yolo_labels_from_coco(list(Path('/kaggle/working/dataset/valid/images').glob("*.jpg")), annotations_dict, valid_labels_dir)

Creating YOLO labels for labels: 100%|██████████| 271/271 [00:00<00:00, 12010.32it/s]


In [6]:
# === YAML ===
data_yaml = f"""
train: /kaggle/working/dataset/train/images
val: /kaggle/working/dataset/valid/images

nc: 5
names:
  0: corrosion
  1: lightning
  2: lightning receptor
  3: missing teeth
  4: patch
"""

with open("/kaggle/working/dataset/data.yaml", "w") as f:
    f.write(data_yaml)

In [ ]:
# === Обучение модели YOLOv11 ===
model = YOLO("/kaggle/input/yolo11s/other/default/1/yolo11s.pt")  # убедись, что yolov11s.pt доступна

config = {
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'fliplr': 0.5,
    'degrees': 15,
    'shear': 0.1,
    'perspective': 0.0001,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.2
}

model.train(data="/kaggle/working/dataset/data.yaml", epochs=100, imgsz=RESCALE_SIZE, batch=4, **config)

Ultralytics 8.3.126 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/data.yaml, degrees=15, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/yolo11s/other/default/1/yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, pers

100%|██████████| 755k/755k [00:00<00:00, 17.5MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.4MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2966.9±1496.9 MB/s, size: 1488.6 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     89/100       5.6G      1.932      1.145     0.9686         31       1024: 100%|██████████| 268/268 [01:02<00:00,  4.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:09<00:00,  3.69it/s]

                   all        271       2619      0.493      0.348      0.357      0.182



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     90/100       5.6G      1.898      1.077     0.9607         52       1024:  10%|█         | 28/268 [00:06<01:04,  3.71it/s]

In [ ]:
model = YOLO("/kaggle/input/100-epochs/other/default/1/last(4).pt")  # убедись, что yolov11s.pt доступна

config = {
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'warmup_momentum': 0.8,
    'fliplr': 0.5,
    'degrees': 15,
    'shear': 0.1,
    'perspective': 0.0001,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'translate': 0.1,
    'scale': 0.2
}

model.train(data="/kaggle/working/dataset/data.yaml", epochs=100, imgsz=RESCALE_SIZE, batch=4, **config)

Ultralytics 8.3.127 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/dataset/data.yaml, degrees=15, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/input/100-epochs/other/default/1/last(4).pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, p

100%|██████████| 755k/755k [00:00<00:00, 16.9MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 5.35M/5.35M [00:00<00:00, 70.0MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2881.1±1596.2 MB/s, size: 1488.6 KB)


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     16/100       5.6G      2.073      1.316     0.9844         30       1024: 100%|██████████| 268/268 [01:03<00:00,  4.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:09<00:00,  3.52it/s]

                   all        271       2619      0.521      0.287      0.333      0.143



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     17/100       5.6G      2.111      1.293       1.03          2       1024: 100%|██████████| 268/268 [01:03<00:00,  4.23it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  47%|████▋     | 16/34 [00:05<00:09,  1.96it/s]

In [9]:
from ultralytics import YOLO
import os

model = YOLO('/kaggle/working/runs/detect/train/weights/last.pt')  # или last.pt

# Путь к тестовым изображениям
test_dir = '/kaggle/working/dataset/test/images'
results = model.predict(source=test_dir, save=False)


image 1/151 /kaggle/working/dataset/test/images/20240120_150729_jpg.rf.5925f54483de8cc52f75fdd1dc0ec3a0.jpg: 1024x1024 (no detections), 20.0ms
image 2/151 /kaggle/working/dataset/test/images/20240120_153628_jpg.rf.67030e3c4592587d26d7762e8e091020.jpg: 1024x1024 (no detections), 19.9ms
image 3/151 /kaggle/working/dataset/test/images/20240120_153658_jpg.rf.66dc04dc4ee8194ccc19397d4ad42f2e.jpg: 1024x1024 (no detections), 19.9ms
image 4/151 /kaggle/working/dataset/test/images/20240120_153711_jpg.rf.3a50aa9daa881abe4296c7f12b03fcf9.jpg: 1024x1024 (no detections), 19.9ms
image 5/151 /kaggle/working/dataset/test/images/20240120_153748-0-_jpg.rf.6c50a8089bf3ccd32bb61e347e9f4019.jpg: 1024x1024 (no detections), 19.9ms
image 6/151 /kaggle/working/dataset/test/images/20240120_153754_jpg.rf.01e099db13ef7c0beee133daa6379466.jpg: 1024x1024 (no detections), 19.9ms
image 7/151 /kaggle/working/dataset/test/images/20240120_153843-0-_jpg.rf.60b55d7dd594d0869bf7216a34d16385.jpg: 1024x1024 (no detections),

In [17]:
import pandas as pd
import json
from pathlib import Path

data = []

for r in results:
    image_name = Path(r.path).name
    objects = []

    for box in r.boxes:
        cls_id = int(box.cls[0])
        label = model.names[cls_id]
        conf = float(box.conf[0])

        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = float(x1), float(y1), float(x2), float(y2)

        w = x2 - x1
        h = y2 - y1
        xc = x1 + w / 2
        yc = y1 + h / 2

        objects.append([label, xc, yc, w, h, conf])

    data.append({
        "image_id": image_name,
        "objects": json.dumps(objects)
    })

In [18]:
df = pd.DataFrame(data)
df.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
model.eval()

In [7]:
import csv
from pathlib import Path
from ultralytics import YOLO
import torch
import pandas as pd

import cv2

model = YOLO("/kaggle/input/200-epochs/other/default/1/last(5).pt")

def get_image_size(image_path):
    img = cv2.imread(str(image_path))
    height, width = img.shape[:2]
    return width, height  # (width, height)

# Загрузка обученной модели
# model = YOLO('best.pt')  # Укажите путь к вашей обученной модели

# Словарь для преобразования ID классов в имена
class_names = {
    0: 'corrosion',
    1: 'lightning',
    2: 'lightning receptor',
    3: 'missing teeth',
    4: 'patch'
}

# Функция для преобразования YOLO формата в требуемый
def format_prediction(box, class_id, conf, orig_width, orig_height):
    x_center = box[0].item() * orig_width
    y_center = box[1].item() * orig_height
    width = box[2].item() * orig_width
    height = box[3].item() * orig_height

    # Вычисляем координаты левого верхнего угла
    x1 = x_center - (width / 2)
    y1 = y_center - (height / 2)
    
    return [
        class_names[class_id],
        round(x1, 2),
        round(y1, 2),
        round(width, 2),
        round(height, 2),
        round(conf.item(), 5)
    ]

# Генерация предсказаний
def generate_predictions(test_dir, prediction_params):
    results = []
    
    # # Новые параметры предсказания
    # prediction_params = {
    #     'conf': 0.01,      # Оставить низкий порог
    #     'iou': 0.45,        # Более строгий NMS
    #     'agnostic_nms': False,
    #     'augment': True,    # TTA
    #     'half': True,       # Использовать половинную точность
    #     'max_det': 300      # Увеличить лимит детекций
    # }
    
    for img_path in tqdm(sorted(Path(test_dir).rglob('*.jpg'))):
        orig_width, orig_height = get_image_size(img_path)
        
        # Предсказание с новыми параметрами
        prediction = model.predict(img_path, **prediction_params)
        
        # Обработка результатов
        boxes = prediction[0].boxes.xywhn
        classes = prediction[0].boxes.cls
        confs = prediction[0].boxes.conf
        
        # Фильтрация слишком больших боксов
        filtered = []
        for box, class_id, conf in zip(boxes, classes, confs):
            w = box[2].item() * orig_width
            h = box[3].item() * orig_height
            
            # Фильтр по размеру (настройте под ваши данные)
            # if 20 < w < orig_width*0.8 and 20 < h < orig_height*0.8:
            filtered.append((box, class_id, conf))
        
        # Форматирование
        formatted = [
            format_prediction(b, int(cid), cf, orig_width, orig_height)
            for b, cid, cf in filtered
        ]
        
        results.append({
            'image_id': img_path.name,
            'objects': formatted
        })
    
    return results

# Генерируем предсказания
test_dir = '/kaggle/working/dataset/test/images'
# predictions = generate_predictions(test_dir)

best_params = {
    'conf': 0.001,       # Низкий порог (v1)
    'iou': 0.2,          # Мягкий NMS (v1)
    'augment': True,     # TTA (v1, v4)
    'half': True         # FP16 (v4)
}

param_combinations  = [
    # Вариант 7: Увеличенный лимит детекций
    # {**best_params, 'max_det': 500},
    
    # Вариант 8: Строгий NMS с TTA
    # {'conf': 0.001, 'iou': 0.5, 'augment': True, 'half': True},
        
    # Вариант 10: Ансамблирование (требует 2+ моделей)
    # {'conf': 0.005, 'iou': 0.4, 'augment': True},

    # Ultra-low confidence
    {'conf': 0.0005, 'iou': 0.2, 'augment': True, 'half': True, 'max_det': 300},
    
    # Balanced params
    {'conf': 0.001, 'iou': 0.25, 'augment': True, 'half': False, 'max_det': 200},
    
    # Size filtering
    {'conf': 0.001, 'iou': 0.2, 'augment': True},
    
    # Hybrid approach
    {'conf': [0.001, 0.01], 'iou': [0.2, 0.4], 'augment': True}  # Мульти-порог
]

for i, params in enumerate(param_combinations, 1):
    print(f"\nГенерация предсказаний с параметрами {params}")
    
    # Добавляем обязательные параметры
    full_params = {
        'agnostic_nms': False,
        **params
    }
    
    predictions = generate_predictions(test_dir, full_params)
    
    # Формируем DataFrame
    images = []
    objects = []
    
    for pred in predictions:
        objects_str = str([[obj[0]] + obj[1:] for obj in pred['objects']]).replace("'", '"')
        objects.append(objects_str)
        images.append(pred['image_id'])
    
    preds = pd.DataFrame({'image_id': images, 'objects': objects})
    
    # Сохраняем файл
    filename = f'submission_v{i}.csv'
    preds.to_csv(filename, index=False)
    print(f"Файл {filename} успешно сохранен")
    
    # Для удобства сохраняем и параметры
    with open(f'params_v{i}.txt', 'w') as f:
        f.write(str(params))


Генерация предсказаний с параметрами {'conf': 0.0005, 'iou': 0.2, 'augment': True, 'half': True, 'max_det': 300}


  0%|          | 0/151 [00:00<?, ?it/s]


image 1/1 /kaggle/working/dataset/test/images/20240120_150729_jpg.rf.5925f54483de8cc52f75fdd1dc0ec3a0.jpg: 1024x1024 3 corrosions, 7 missing teeths, 223.6ms
Speed: 17.4ms preprocess, 223.6ms inference, 288.6ms postprocess per image at shape (1, 3, 1024, 1024)


  1%|          | 1/151 [00:01<04:55,  1.97s/it]


image 1/1 /kaggle/working/dataset/test/images/20240120_153628_jpg.rf.67030e3c4592587d26d7762e8e091020.jpg: 1024x1024 3 corrosions, 11 missing teeths, 40.9ms
Speed: 11.5ms preprocess, 40.9ms inference, 1.8ms postprocess per image at shape (1, 3, 1024, 1024)


...


image 1/1 /kaggle/working/dataset/test/images/DJI_0997_JPG.rf.9f5f2b599b0927db0d1121a39516f4e0.jpg: 576x1024 56 corrosions, 4 missing teeths, 30.4ms
Speed: 4.5ms preprocess, 30.4ms inference, 1.4ms postprocess per image at shape (1, 3, 576, 1024)


100%|██████████| 151/151 [00:20<00:00,  7.29it/s]


Файл submission_v3.csv успешно сохранен

Генерация предсказаний с параметрами {'conf': [0.001, 0.01], 'iou': [0.2, 0.4], 'augment': True}


  0%|          | 0/151 [00:00<?, ?it/s]


TypeError: 'conf=[0.001, 0.01]' is of invalid type list. Valid 'conf' types are int (i.e. 'conf=0') or float (i.e. 'conf=0.5')

In [8]:
import csv
from pathlib import Path
from ultralytics import YOLO
import torch
import pandas as pd

import cv2

model = YOLO("/kaggle/input/best-of-200-epochs/other/default/1/best(5).pt")

def get_image_size(image_path):
    img = cv2.imread(str(image_path))
    height, width = img.shape[:2]
    return width, height  # (width, height)

# Загрузка обученной модели
# model = YOLO('best.pt')  # Укажите путь к вашей обученной модели

# Словарь для преобразования ID классов в имена
class_names = {
    0: 'corrosion',
    1: 'lightning',
    2: 'lightning receptor',
    3: 'missing teeth',
    4: 'patch'
}

# Функция для преобразования YOLO формата в требуемый
def format_prediction(box, class_id, conf, orig_width, orig_height):
    x_center = box[0].item() * orig_width
    y_center = box[1].item() * orig_height
    width = box[2].item() * orig_width
    height = box[3].item() * orig_height

    # Вычисляем координаты левого верхнего угла
    x1 = x_center - (width / 2)
    y1 = y_center - (height / 2)
    
    return [
        class_names[class_id],
        round(x1, 2),
        round(y1, 2),
        round(width, 2),
        round(height, 2),
        round(conf.item(), 5)
    ]

# Генерация предсказаний
def generate_predictions(test_dir, prediction_params):
    results = []
    
    # # Новые параметры предсказания
    # prediction_params = {
    #     'conf': 0.01,      # Оставить низкий порог
    #     'iou': 0.45,        # Более строгий NMS
    #     'agnostic_nms': False,
    #     'augment': True,    # TTA
    #     'half': True,       # Использовать половинную точность
    #     'max_det': 300      # Увеличить лимит детекций
    # }
    
    for img_path in tqdm(sorted(Path(test_dir).rglob('*.jpg'))):
        orig_width, orig_height = get_image_size(img_path)
        
        # Предсказание с новыми параметрами
        prediction = model.predict(img_path, **prediction_params)
        
        # Обработка результатов
        boxes = prediction[0].boxes.xywhn
        classes = prediction[0].boxes.cls
        confs = prediction[0].boxes.conf
        
        # Фильтрация слишком больших боксов
        filtered = []
        for box, class_id, conf in zip(boxes, classes, confs):
            w = box[2].item() * orig_width
            h = box[3].item() * orig_height
            
            # Фильтр по размеру (настройте под ваши данные)
            # if 20 < w < orig_width*0.8 and 20 < h < orig_height*0.8:
            filtered.append((box, class_id, conf))
        
        # Форматирование
        formatted = [
            format_prediction(b, int(cid), cf, orig_width, orig_height)
            for b, cid, cf in filtered
        ]
        
        results.append({
            'image_id': img_path.name,
            'objects': formatted
        })
    
    return results

# Генерируем предсказания
test_dir = '/kaggle/working/dataset/test/images'
# predictions = generate_predictions(test_dir)

best_params = {
    'conf': 0.001,       # Низкий порог (v1)
    'iou': 0.2,          # Мягкий NMS (v1)
    'augment': True,     # TTA (v1, v4)
    'half': True         # FP16 (v4)
}

param_combinations  = [
    # Вариант 7: Увеличенный лимит детекций
    # {**best_params, 'max_det': 500},
    
    # Вариант 8: Строгий NMS с TTA
    # {'conf': 0.001, 'iou': 0.5, 'augment': True, 'half': True},
        
    # Вариант 10: Ансамблирование (требует 2+ моделей)
    # {'conf': 0.005, 'iou': 0.4, 'augment': True},

    # Ultra-low confidence
    {'conf': 0.0005, 'iou': 0.2, 'augment': True, 'half': True, 'max_det': 300},
    
    # Balanced params
    {'conf': 0.001, 'iou': 0.25, 'augment': True, 'half': False, 'max_det': 200},
    
    # Size filtering
    {'conf': 0.001, 'iou': 0.2, 'augment': True},
    
    # Hybrid approach
    {'conf': [0.001, 0.01], 'iou': [0.2, 0.4], 'augment': True}  # Мульти-порог
]

for i, params in enumerate(param_combinations, 1):
    print(f"\nГенерация предсказаний с параметрами {params}")
    
    # Добавляем обязательные параметры
    full_params = {
        'agnostic_nms': False,
        **params
    }
    
    predictions = generate_predictions(test_dir, full_params)
    
    # Формируем DataFrame
    images = []
    objects = []
    
    for pred in predictions:
        objects_str = str([[obj[0]] + obj[1:] for obj in pred['objects']]).replace("'", '"')
        objects.append(objects_str)
        images.append(pred['image_id'])
    
    preds = pd.DataFrame({'image_id': images, 'objects': objects})
    
    # Сохраняем файл
    filename = f'new_submission_v{i}.csv'
    preds.to_csv(filename, index=False)
    print(f"Файл {filename} успешно сохранен")
    
    # Для удобства сохраняем и параметры
    with open(f'new_params_v{i}.txt', 'w') as f:
        f.write(str(params))


Генерация предсказаний с параметрами {'conf': 0.0005, 'iou': 0.2, 'augment': True, 'half': True, 'max_det': 300}


  0%|          | 0/151 [00:00<?, ?it/s]


image 1/1 /kaggle/working/dataset/test/images/20240120_150729_jpg.rf.5925f54483de8cc52f75fdd1dc0ec3a0.jpg: 1024x1024 12 corrosions, 5 missing teeths, 40.6ms
Speed: 7.4ms preprocess, 40.6ms inference, 1.5ms postprocess per image at shape (1, 3, 1024, 1024)


  1%|          | 1/151 [00:00<00:50,  2.98it/s]


image 1/1 /kaggle/working/dataset/test/images/20240120_153628_jpg.rf.67030e3c4592587d26d7762e8e091020.jpg: 1024x1024 14 corrosions, 1 lightning receptor, 1 missing teeth, 1 patch, 41.1ms
Speed: 8.5ms preprocess, 41.1ms inference, 1.4ms postprocess per image at shape (1, 3, 1024, 1024)


...


image 1/1 /kaggle/working/dataset/test/images/DJI_0997_JPG.rf.9f5f2b599b0927db0d1121a39516f4e0.jpg: 576x1024 70 corrosions, 8 missing teeths, 30.7ms
Speed: 4.7ms preprocess, 30.7ms inference, 1.5ms postprocess per image at shape (1, 3, 576, 1024)


100%|██████████| 151/151 [00:26<00:00,  5.80it/s]


Файл new_submission_v3.csv успешно сохранен

Генерация предсказаний с параметрами {'conf': [0.001, 0.01], 'iou': [0.2, 0.4], 'augment': True}


  0%|          | 0/151 [00:00<?, ?it/s]


TypeError: 'conf=[0.001, 0.01]' is of invalid type list. Valid 'conf' types are int (i.e. 'conf=0') or float (i.e. 'conf=0.5')

In [ ]:
images = []
objects = []

for pred in predictions:
    # Форматируем список объектов в строку
    objects_str = str([[obj[0]] + obj[1:] for obj in pred['objects']])\
        .replace("'", '"')
    objects.append(objects_str)
    
    images.append(pred['image_id'])

preds = pd.DataFrame({'image_id': images,
                      'objects': objects})